# BBC News Heading

In [11]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm

In [2]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

In [33]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
splits = {'train': 'bbc hindi nli/train-00000-of-00001.parquet', 'validation': 'bbc hindi nli/validation-00000-of-00001.parquet', 'test': 'bbc hindi nli/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]


In [34]:
train_texts, train_labels,test_texts, test_labels = df['premise'].tolist(), df['topic'].tolist(), test_df['premise'].tolist(), test_df['topic'].tolist()

In [36]:
splits = {'train': 'bbc hindi nli/train-00000-of-00001.parquet', 'validation': 'bbc hindi nli/validation-00000-of-00001.parquet', 'test': 'bbc hindi nli/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]
df.drop_duplicates()
test_df.drop_duplicates()

,premise,topic
0,बर्मा में मुसलमानों के खिलाफ हिंसा के और सुबूत,2
12,चीन: कपड़े पहनो या जेल जाओ,2
24,अन्ना की 'जनतंत्र यात्रा' शुरू,0
36,किशोरों का शैक्षणिक प्रदर्शन बेहतर बनाता कसरत,5
48,तेलंगाना राज्य के लिए छात्र ने दी जान,0
...,...,...
2520,भारत: बाढ़ से असम के 14 ज़िले प्रभावित,0
2532,अफगान लड़की की 'चीखती तस्वीर' को पुलित्जर,1
2544,ब्रिटेन के चर्चित प्रसारक सर डेविड फ़्रॉस्ट नह...,2
2556,नसीर के साथ सनी लियोनी गोवा में,3


In [39]:
len(set(df.drop_duplicates()['topic']))

6

## Muril
Best accuracy 76.27 with 2e-5 as lr in 4 epochs. Lowest loss 0.7901

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
splits = {'train': 'bbc hindi nli/train-00000-of-00001.parquet', 'validation': 'bbc hindi nli/validation-00000-of-00001.parquet', 'test': 'bbc hindi nli/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]



import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.nn.functional import cross_entropy

# Step 1: Load and preprocess data
def prepare_data(df, tokenizer, max_length=128):
    encodings = tokenizer(list(df["premise"]), truncation=True, padding=True, max_length=max_length)
    labels = torch.tensor(list(df["topic"]))
    dataset = torch.utils.data.TensorDataset(
        torch.tensor(encodings["input_ids"]), 
        torch.tensor(encodings["attention_mask"]), 
        labels
    )
    return dataset

# Step 2: Define training function
def train_model(model, train_loader, val_loader, optimizer, device, epochs=50):
    model.to(device)
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                # print(labels)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
                predictions = torch.argmax(outputs.logits, dim=-1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")


train_df = df.drop_duplicates()
val_df = test_df.drop_duplicates()
# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
# google/muril-base-cased
train_dataset = prepare_data(train_df, tokenizer)
val_dataset = prepare_data(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=6)
model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=6)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

train_model(model, train_loader, val_loader, optimizer, device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 1.6128714926971877
Epoch 1, Validation Loss: 1.3402898907661438, Accuracy: 0.6232558139534884
Epoch 2, Training Loss: 1.2266160370397174
Epoch 2, Validation Loss: 0.9993197066443307, Accuracy: 0.7116279069767442
Epoch 3, Training Loss: 0.9520994072610681
Epoch 3, Validation Loss: 0.937669004712786, Accuracy: 0.6790697674418604
Epoch 4, Training Loss: 0.7598672132354137
Epoch 4, Validation Loss: 0.7901706184659686, Accuracy: 0.7627906976744186
Epoch 5, Training Loss: 0.6034676263278181
Epoch 5, Validation Loss: 0.9669486199106488, Accuracy: 0.6883720930232559
Epoch 6, Training Loss: 0.5040924563572919
Epoch 6, Validation Loss: 0.799317581312997, Accuracy: 0.7488372093023256
Epoch 7, Training Loss: 0.4249426610152091
Epoch 7, Validation Loss: 0.8545383640698024, Accuracy: 0.7395348837209302
Epoch 8, Training Loss: 0.35268002702308093
Epoch 8, Validation Loss: 0.9450189343520573, Accuracy: 0.7162790697674418
Epoch 9, Training Loss: 0.29075694886182457
Epoch 9, Vali


KeyboardInterrupt



## Indic BERT
Best accuracy 52.55 with 8e-6 as lr in 39 epoch and loss 2.608. Lowest loss 1.391 in epoch 4

In [8]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
splits = {'train': 'bbc hindi nli/train-00000-of-00001.parquet', 'validation': 'bbc hindi nli/validation-00000-of-00001.parquet', 'test': 'bbc hindi nli/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]



import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.nn.functional import cross_entropy

# Step 1: Load and preprocess data
def prepare_data(df, tokenizer, max_length=128):
    encodings = tokenizer(list(df["premise"]), truncation=True, padding=True, max_length=max_length)
    labels = torch.tensor(list(df["topic"]))
    dataset = torch.utils.data.TensorDataset(
        torch.tensor(encodings["input_ids"]), 
        torch.tensor(encodings["attention_mask"]), 
        labels
    )
    return dataset

# Step 2: Define training function
def train_model(model, train_loader, val_loader, optimizer, device, epochs=50):
    model.to(device)
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
                predictions = torch.argmax(outputs.logits, dim=-1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")


train_df = df.drop_duplicates()
val_df = test_df.drop_duplicates()
# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
# google/muril-base-cased
train_dataset = prepare_data(train_df, tokenizer)
val_dataset = prepare_data(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=6)
# model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/IndicBART", num_labels=6)
optimizer = torch.optim.AdamW(model.parameters(), lr=8e-6)
#BEST 6e-6
train_model(model, train_loader, val_loader, optimizer, device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu"))

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 1.5721416172902447
Epoch 1, Validation Loss: 1.4362329159464156, Accuracy: 0.40930232558139534
Epoch 2, Training Loss: 1.4714490140765166
Epoch 2, Validation Loss: 1.4179339238575526, Accuracy: 0.40930232558139534
Epoch 3, Training Loss: 1.4343787744518155
Epoch 3, Validation Loss: 1.3910181181771415, Accuracy: 0.4418604651162791
Epoch 4, Training Loss: 1.3845286216617616
Epoch 4, Validation Loss: 1.3617682797568185, Accuracy: 0.44651162790697674
Epoch 5, Training Loss: 1.319368320063126
Epoch 5, Validation Loss: 1.3907011151313782, Accuracy: 0.44651162790697674
Epoch 6, Training Loss: 1.2357329031652655
Epoch 6, Validation Loss: 1.4082022394452776, Accuracy: 0.4604651162790698
Epoch 7, Training Loss: 1.116824305747166
Epoch 7, Validation Loss: 1.3923543521336146, Accuracy: 0.4697674418604651
Epoch 8, Training Loss: 1.0290538850894644
Epoch 8, Validation Loss: 1.469522476196289, Accuracy: 0.5069767441860465
Epoch 9, Training Loss: 0.9087859076901901
Epoch 9, Val

## Indic Bert-MLM v2
Epoch 8, Validation Loss: 0.7636773117950985, Accuracy: 0.7767441860465116. Lowest loss 0.7081 in 4 epochs

In [ ]:
## 77% for 4e-6 in 8th epoch

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm

splits = {'train': 'bbc hindi nli/train-00000-of-00001.parquet', 'validation': 'bbc hindi nli/validation-00000-of-00001.parquet', 'test': 'bbc hindi nli/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]



import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.nn.functional import cross_entropy

# Step 1: Load and preprocess data
def prepare_data(df, tokenizer, max_length=128):
    encodings = tokenizer(list(df["premise"]), truncation=True, padding=True, max_length=max_length)
    labels = torch.tensor(list(df["topic"]))
    dataset = torch.utils.data.TensorDataset(
        torch.tensor(encodings["input_ids"]), 
        torch.tensor(encodings["attention_mask"]), 
        labels
    )
    return dataset

# Step 2: Define training function
def train_model(model, train_loader, val_loader, optimizer, device, epochs=50):
    model.to(device)
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
                predictions = torch.argmax(outputs.logits, dim=-1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")


train_df = df.drop_duplicates()
val_df = test_df.drop_duplicates()
# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-MLM-Sam-TLM")
# google/muril-base-cased
train_dataset = prepare_data(train_df, tokenizer)
val_dataset = prepare_data(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/IndicBERTv2-MLM-Sam-TLM", num_labels=6)

optimizer = torch.optim.AdamW(model.parameters(), lr=6e-6)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.98)
train_model(model, train_loader, val_loader, optimizer, device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu"))
# train_model(model, train_loader, val_loader, optimizer, scheduler, device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-Sam-TLM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 1.528518676265212
Epoch 1, Validation Loss: 1.18868972999709, Accuracy: 0.5813953488372093
Epoch 2, Training Loss: 1.0532700815969263
Epoch 2, Validation Loss: 0.8067211636475154, Accuracy: 0.7162790697674418
Epoch 3, Training Loss: 0.7723936894215828
Epoch 3, Validation Loss: 0.7482050742421832, Accuracy: 0.7627906976744186
Epoch 4, Training Loss: 0.6588754484599287
Epoch 4, Validation Loss: 0.7081855876105172, Accuracy: 0.7488372093023256
Epoch 5, Training Loss: 0.5905470670131613
Epoch 5, Validation Loss: 0.7083919963666371, Accuracy: 0.7488372093023256
Epoch 6, Training Loss: 0.524846112863584
Epoch 6, Validation Loss: 0.7408184962613242, Accuracy: 0.7581395348837209
Epoch 7, Training Loss: 0.4802014953957116
Epoch 7, Validation Loss: 0.7467416312013354, Accuracy: 0.7627906976744186
Epoch 8, Training Loss: 0.4322499161170534
Epoch 8, Validation Loss: 0.7636773117950985, Accuracy: 0.7767441860465116
Epoch 9, Training Loss: 0.3942678075376128
Epoch 9, Validati

Traceback (most recent call last):
  File "/home/arnabdey/miniconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2401929/486726467.py", line 124, in <module>
    train_model(model, train_loader, val_loader, optimizer, device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu"))
  File "/tmp/ipykernel_2401929/486726467.py", line 47, in train_model
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/arnabdey/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/arnabdey/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forwar

KeyboardInterrupt: 

## Roberta-Base
Epoch 7, Validation Loss: 0.8164238972323281, Accuracy: 0.7162790697674418

In [ ]:
model_name = "FacebookAI/xlm-roberta-base"

## 77% for 4e-6 in 8th epoch

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm

splits = {'train': 'bbc hindi nli/train-00000-of-00001.parquet', 'validation': 'bbc hindi nli/validation-00000-of-00001.parquet', 'test': 'bbc hindi nli/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]



import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.nn.functional import cross_entropy

# Step 1: Load and preprocess data
def prepare_data(df, tokenizer, max_length=128):
    encodings = tokenizer(list(df["premise"]), truncation=True, padding=True, max_length=max_length)
    labels = torch.tensor(list(df["topic"]))
    dataset = torch.utils.data.TensorDataset(
        torch.tensor(encodings["input_ids"]), 
        torch.tensor(encodings["attention_mask"]), 
        labels
    )
    return dataset

# Step 2: Define training function
def train_model(model, train_loader, val_loader, optimizer, device, epochs=50):
    model.to(device)
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
                predictions = torch.argmax(outputs.logits, dim=-1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")


train_df = df.drop_duplicates()
val_df = test_df.drop_duplicates()
# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
tokenizer = AutoTokenizer.from_pretrained(model_name)
# google/muril-base-cased
train_dataset = prepare_data(train_df, tokenizer)
val_dataset = prepare_data(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-6)
train_model(model, train_loader, val_loader, optimizer, device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu"))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 1.5341656079962234
Epoch 1, Validation Loss: 1.2197813732283456, Accuracy: 0.5116279069767442
Epoch 2, Training Loss: 1.2222807205412998
Epoch 2, Validation Loss: 1.0273845408643996, Accuracy: 0.6465116279069767
Epoch 3, Training Loss: 1.0437555832803742
Epoch 3, Validation Loss: 0.8941920782838549, Accuracy: 0.6744186046511628
Epoch 4, Training Loss: 0.9362325711437494
Epoch 4, Validation Loss: 0.9143130012920925, Accuracy: 0.6744186046511628
Epoch 5, Training Loss: 0.8752147281958051
Epoch 5, Validation Loss: 0.840754074709756, Accuracy: 0.6930232558139535
Epoch 6, Training Loss: 0.8208065419650274
Epoch 6, Validation Loss: 0.8333501858370644, Accuracy: 0.6976744186046512
Epoch 7, Training Loss: 0.7756790883284955
Epoch 7, Validation Loss: 0.8164238972323281, Accuracy: 0.7162790697674418
Epoch 8, Training Loss: 0.7049780963373579
Epoch 8, Validation Loss: 0.8785512511219297, Accuracy: 0.6976744186046512
Epoch 9, Training Loss: 0.6677530559380193
Epoch 9, Valid

KeyboardInterrupt: 

# BBC News Article

In [40]:
df_orig = pd.read_json("bbc-hindi-news.json")

df_orig['premise'] =  df_orig["body"] + "\n" + df_orig["heading"] + "\n" + df_orig["intro"]
from collections import defaultdict
mapper = defaultdict()
for i,a in enumerate(set(df_orig['tag'])):
    mapper[a] = i
df_orig['topic'] = [mapper[a] for a in df_orig['tag']]

df, test_df = train_test_split(df_orig, test_size=0.2, random_state=42)
# df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
# test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]

### Muril

In [1]:
# pd.read_json("bbc-hindi-news.json")
model_name = "google/muril-base-cased"

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
df_orig = pd.read_json("bbc-hindi-news.json")

df_orig['premise'] =  df_orig["body"] + "\n" + df_orig["heading"] + "\n" + df_orig["intro"]
from collections import defaultdict
mapper = defaultdict()
for i,a in enumerate(set(df_orig['tag'])):
    mapper[a] = i
df_orig['topic'] = [mapper[a] for a in df_orig['tag']]

df, test_df = train_test_split(df_orig, test_size=0.2, random_state=42)
# df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
# test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]


import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.nn.functional import cross_entropy

# Step 1: Load and preprocess data
def prepare_data(df, tokenizer, max_length=128):
    encodings = tokenizer(list(df["premise"]), truncation=True, padding=True, max_length=max_length)
    labels = torch.tensor(list(df["topic"]))
    dataset = torch.utils.data.TensorDataset(
        torch.tensor(encodings["input_ids"]), 
        torch.tensor(encodings["attention_mask"]), 
        labels
    )
    return dataset

# Step 2: Define training function
def train_model(model, train_loader, val_loader, optimizer, device, epochs=10):
    model.to(device)
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
                predictions = torch.argmax(outputs.logits, dim=-1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")


train_df = df.drop_duplicates()
val_df = test_df.drop_duplicates()
# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
tokenizer = AutoTokenizer.from_pretrained(model_name)
# google/muril-base-cased
train_dataset = prepare_data(train_df, tokenizer)
val_dataset = prepare_data(val_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=6)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=14)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# train_model(model, train_loader, val_loader, optimizer, device=torch.device("cpu"))
train_model(model, train_loader, val_loader, optimizer, device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 2.3515591830152522
Epoch 1, Validation Loss: 1.9448187676343052, Accuracy: 0.4209919261822376
Epoch 2, Training Loss: 1.7984719490675334
Epoch 2, Validation Loss: 1.6147457166151566, Accuracy: 0.4209919261822376
Epoch 3, Training Loss: 1.5185726186097492
Epoch 3, Validation Loss: 1.358221820267764, Accuracy: 0.6113033448673587
Epoch 4, Training Loss: 1.2636147212872308
Epoch 4, Validation Loss: 1.2332896037535235, Accuracy: 0.7162629757785467
Epoch 5, Training Loss: 1.065673594650585
Epoch 5, Validation Loss: 1.1672565070065586, Accuracy: 0.7185697808535179
Epoch 6, Training Loss: 0.9343058994288818
Epoch 6, Validation Loss: 1.1324865498326042, Accuracy: 0.71280276816609
Epoch 7, Training Loss: 0.8022306705674818
Epoch 7, Validation Loss: 1.0720029072328048, Accuracy: 0.7174163783160323
Epoch 8, Training Loss: 0.7128771581270728
Epoch 8, Validation Loss: 1.081256899508563, Accuracy: 0.7474048442906575
Epoch 9, Training Loss: 0.6243687025893668
Epoch 9, Validatio


KeyboardInterrupt



In [ ]:
# pd.read_json("bbc-hindi-news.json")
model_name = "google/muril-base-cased"

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
df_orig = pd.read_json("bbc-hindi-news.json")

df_orig['premise'] =  df_orig["body"] + "\n" + df_orig["heading"] + "\n" + df_orig["intro"]
from collections import defaultdict
mapper = defaultdict()
for i,a in enumerate(set(df_orig['tag'])):
    mapper[a] = i
df_orig['topic'] = [mapper[a] for a in df_orig['tag']]

df, test_df = train_test_split(df_orig, test_size=0.2, random_state=42)
# df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["train"])
# test_df = pd.read_parquet("hf://datasets/midas/bbc_hindi_nli/" + splits["validation"])
df = df[['premise','topic']]
test_df = test_df[['premise','topic']]


import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.nn.functional import cross_entropy


def prepare_data(df, tokenizer, max_length=128):
    encodings = tokenizer(list(df["premise"]), truncation=True, padding=True, max_length=max_length)
    labels = torch.tensor(list(df["topic"]))
    dataset = torch.utils.data.TensorDataset(
        torch.tensor(encodings["input_ids"]), 
        torch.tensor(encodings["attention_mask"]), 
        labels
    )
    return dataset

# Step 2: Define training function
def train_model(model, train_loader, val_loader, optimizer, device, epochs=15):
    model.to(device)
    best_val_accuracy = -1
    best_loss = 200
    best_acc_epoch = -1
    best_loss_epoch = -1
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
                predictions = torch.argmax(outputs.logits, dim=-1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")
        if val_loss / len(val_loader) < best_loss:
            best_loss = val_loss / len(val_loader)
            best_loss_epoch = epoch
        if correct / total > best_val_accuracy:
            best_val_accuracy = correct / total
            best_acc_epoch = epoch
    print(f"Model {model_name}Best Accuracy: {best_val_accuracy} at epoch {best_acc_epoch}, Best Loss:{best_loss} at epoch {best_loss_epoch}")
datasets = ["FacebookAI/xlm-roberta-base","ai4bharat/indic-bert","ai4bharat/IndicBERTv2-MLM-Sam-TLM", "google/muril-base-cased","google-bert/bert-base-multilingual-cased"]
for model_name in datasets:
    print(f"###################{model_name}#####################")
    train_df = df.drop_duplicates()
    val_df = test_df.drop_duplicates()
    # tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # google/muril-base-cased
    train_dataset = prepare_data(train_df, tokenizer)
    val_dataset = prepare_data(val_df, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)
    
    # model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=6)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=14)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    
    # train_model(model, train_loader, val_loader, optimizer, device=torch.device("cpu"))
    train_model(model, train_loader, val_loader, optimizer, device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu"))
    model.detach().cpu()
    del model
    torch.cuda.empty_cache()

###################FacebookAI/xlm-roberta-base#####################


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 1.5901823865103832
Epoch 1, Validation Loss: 1.0513156609101728, Accuracy: 0.6793540945790081
Epoch 2, Training Loss: 0.9788123694707721
Epoch 2, Validation Loss: 0.9561538837172768, Accuracy: 0.7393310265282583
Epoch 3, Training Loss: 0.8483821810115867
Epoch 3, Validation Loss: 0.8902791500091553, Accuracy: 0.7508650519031141
Epoch 4, Training Loss: 0.7423257741922608
Epoch 4, Validation Loss: 0.8930524874817242, Accuracy: 0.76239907727797
Epoch 5, Training Loss: 0.6820481074158498
Epoch 5, Validation Loss: 0.9524736984209581, Accuracy: 0.7381776239907728
Epoch 6, Training Loss: 0.6007020544209238
Epoch 6, Validation Loss: 0.9523431861942465, Accuracy: 0.7439446366782007


# IIT Patna Reviews

In [28]:
# pd.read_json("bbc-hindi-news.json")
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.nn.functional import cross_entropy


def prepare_data(df, tokenizer, max_length=128):
    encodings = tokenizer(list(df["premise"]), truncation=True, padding=True, max_length=max_length)
    labels = torch.tensor(list(df["topic"]))
    dataset = torch.utils.data.TensorDataset(
        torch.tensor(encodings["input_ids"]), 
        torch.tensor(encodings["attention_mask"]), 
        labels
    )
    return dataset

# Step 2: Define training function
def train_model(model, train_loader, val_loader, optimizer,num_labels, device, epochs=10):
    model.to(device)
    best_val_accuracy = -1
    best_loss = 200
    best_acc_epoch = -1
    best_loss_epoch = -1
    class_report = None
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
                predictions = torch.argmax(outputs.logits, dim=-1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")
        if val_loss / len(val_loader) < best_loss:
            best_loss = val_loss / len(val_loader)
            best_loss_epoch = epoch
        if correct / total > best_val_accuracy:
            best_val_accuracy = correct / total
            best_acc_epoch = epoch
            # class_report = classification_report(labels.cpu(), predictions.cpu(), target_names=[0,1,2])
    
    print(f"Model {model_name}::Best Accuracy: {best_val_accuracy} at epoch {best_acc_epoch}, Best Loss:{best_loss} at epoch {best_loss_epoch}")
    # print(class_report)
    return f"Model {model_name}::Best Accuracy: {best_val_accuracy} at epoch {best_acc_epoch}, Best Loss:{best_loss} at epoch {best_loss_epoch}"

models_list = ["FacebookAI/xlm-roberta-base","ai4bharat/indic-bert","ai4bharat/IndicBERTv2-MLM-Sam-TLM", "google/muril-base-cased","google-bert/bert-base-multilingual-cased"]
res = []
mapper = {'positive': 0, 'negative': 1, 'neutral': 2}
def load_movie_data(file_path, mapper):
    df = pd.read_csv(file_path, header=None, names=['sentiment', 'premise'])
    df['topic'] = [mapper[a] for a in df['sentiment']]
    return df
train_movie = load_movie_data('iitp-movie-reviews/hi/hi-train.csv',mapper)
train_product = load_movie_data('iitp-product-reviews/hi/hi-train.csv',mapper)
test_movie = load_movie_data('iitp-movie-reviews/hi/hi-valid.csv',mapper)
test_product = load_movie_data('iitp-product-reviews/hi/hi-valid.csv',mapper)
test_df =  pd.concat([test_movie, test_product], ignore_index=True)
df = pd.concat([train_movie, train_product], ignore_index=True)
for model_name in models_list:
    print(f"###################{model_name}#####################")
    train_df = df.drop_duplicates()
    val_df = test_df.drop_duplicates()
    # tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # google/muril-base-cased
    train_dataset = prepare_data(train_df, tokenizer)
    val_dataset = prepare_data(val_df, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=6)
    num_labels = 3
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5,  eps=1e-8)
    
    # train_model(model, train_loader, val_loader, optimizer, device=torch.device("cpu"))
    
    res.append(train_model(model, train_loader, val_loader, optimizer,num_labels, device=torch.device("cuda:2" if torch.cuda.is_available() else "cpu")))
    del model
    torch.cuda.empty_cache()
print(res)

###################FacebookAI/xlm-roberta-base#####################


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 0.92800011829688
Epoch 1, Validation Loss: 0.6814157747560077, Accuracy: 0.7046818727490997
Epoch 2, Training Loss: 0.6947922993164796
Epoch 2, Validation Loss: 0.6351744764380984, Accuracy: 0.7190876350540216
Epoch 3, Training Loss: 0.5865695072481265
Epoch 3, Validation Loss: 0.6433643417570878, Accuracy: 0.7274909963985594
Epoch 4, Training Loss: 0.4926654960100467
Epoch 4, Validation Loss: 0.6270273794868478, Accuracy: 0.7430972388955582
Epoch 5, Training Loss: 0.43140838080300736
Epoch 5, Validation Loss: 0.7454673368138848, Accuracy: 0.7178871548619448
Epoch 6, Training Loss: 0.347380382510332
Epoch 6, Validation Loss: 0.703889455249602, Accuracy: 0.7346938775510204
Epoch 7, Training Loss: 0.3018630865531472
Epoch 7, Validation Loss: 0.7736905486564394, Accuracy: 0.7262905162064826
Epoch 8, Training Loss: 0.23611295294876283
Epoch 8, Validation Loss: 0.8156580332390688, Accuracy: 0.7298919567827131
Epoch 9, Training Loss: 0.21168403671338007
Epoch 9, Valid

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 1.035265316756872
Epoch 1, Validation Loss: 1.0091195128582142, Accuracy: 0.46938775510204084
Epoch 2, Training Loss: 0.9524469925807073
Epoch 2, Validation Loss: 0.8882554648099122, Accuracy: 0.5138055222088835
Epoch 3, Training Loss: 0.8480335542788873
Epoch 3, Validation Loss: 0.8095258893790068, Accuracy: 0.602641056422569
Epoch 4, Training Loss: 0.7294597109922996
Epoch 4, Validation Loss: 0.7761878608553497, Accuracy: 0.631452581032413
Epoch 5, Training Loss: 0.6251887942735965
Epoch 5, Validation Loss: 0.800586501205409, Accuracy: 0.6326530612244898
Epoch 6, Training Loss: 0.5153534057048651
Epoch 6, Validation Loss: 0.8814750998108475, Accuracy: 0.6350540216086434
Epoch 7, Training Loss: 0.4111874285512246
Epoch 7, Validation Loss: 0.9234414311401822, Accuracy: 0.6278511404561825
Epoch 8, Training Loss: 0.28593007983783114
Epoch 8, Validation Loss: 1.1436316666572734, Accuracy: 0.6386554621848739
Epoch 9, Training Loss: 0.21986764970307165
Epoch 9, Valid

...

Epoch 1, Training Loss: 0.8526875723439914
Epoch 1, Validation Loss: 0.6436294264125603, Accuracy: 0.7142857142857143
Epoch 2, Training Loss: 0.5561015757803733
Epoch 2, Validation Loss: 0.5895790857159429, Accuracy: 0.7418967587034814
Epoch 3, Training Loss: 0.4557310617887057
Epoch 3, Validation Loss: 0.6109827982813671, Accuracy: 0.7406962785114045
Epoch 4, Training Loss: 0.3837117481117065
Epoch 4, Validation Loss: 0.6453030393976305, Accuracy: 0.7442977190876351
Epoch 5, Training Loss: 0.3062401077208611
Epoch 5, Validation Loss: 0.7280654359815849, Accuracy: 0.7334933973589436
Epoch 6, Training Loss: 0.24135968096267718
Epoch 6, Validation Loss: 0.7876245650911221, Accuracy: 0.7430972388955582
Epoch 7, Training Loss: 0.18432422190044934
Epoch 7, Validation Loss: 0.909902375743345, Accuracy: 0.7274909963985594
Epoch 8, Training Loss: 0.14772622709950575
Epoch 8, Validation Loss: 0.9557134529437732, Accuracy: 0.7298919567827131
Epoch 9, Training Loss: 0.11866855250599866
Epoch 9, V

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 1.050402894616127
Epoch 1, Validation Loss: 0.9813501437505087, Accuracy: 0.5498199279711885
Epoch 2, Training Loss: 0.8810367567034868
Epoch 2, Validation Loss: 0.7935225698682997, Accuracy: 0.7166866746698679
Epoch 3, Training Loss: 0.6947884892041867
Epoch 3, Validation Loss: 0.738339900970459, Accuracy: 0.6818727490996399
Epoch 4, Training Loss: 0.5795039030221792
Epoch 4, Validation Loss: 0.6404747239969395, Accuracy: 0.7346938775510204
Epoch 5, Training Loss: 0.4766646953156361
Epoch 5, Validation Loss: 0.7106503032975726, Accuracy: 0.7262905162064826
Epoch 6, Training Loss: 0.3967226477196583
Epoch 6, Validation Loss: 0.673379375702805, Accuracy: 0.7418967587034814
Epoch 7, Training Loss: 0.3230292640912991
Epoch 7, Validation Loss: 0.7128348993482413, Accuracy: 0.7358943577430972
Epoch 8, Training Loss: 0.2618845035680212
Epoch 8, Validation Loss: 0.7386468230850167, Accuracy: 0.7310924369747899
Epoch 9, Training Loss: 0.22194376294142926
Epoch 9, Valida

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 0.8900154960843233
Epoch 1, Validation Loss: 0.7717706174762161, Accuracy: 0.6506602641056423
Epoch 2, Training Loss: 0.7047240596551162
Epoch 2, Validation Loss: 0.7052453116134361, Accuracy: 0.6890756302521008
Epoch 3, Training Loss: 0.5697525120698489
Epoch 3, Validation Loss: 0.7856275242649846, Accuracy: 0.6818727490996399
Epoch 4, Training Loss: 0.46795985159965664
Epoch 4, Validation Loss: 0.7253206802394103, Accuracy: 0.6986794717887155
Epoch 5, Training Loss: 0.36928142234683037
Epoch 5, Validation Loss: 0.8856774092031022, Accuracy: 0.6938775510204082
Epoch 6, Training Loss: 0.2938302720252138
Epoch 6, Validation Loss: 0.9355454490761514, Accuracy: 0.687875150060024
Epoch 7, Training Loss: 0.23101812980782527
Epoch 7, Validation Loss: 0.9523165012727043, Accuracy: 0.6998799519807923
Epoch 8, Training Loss: 0.17537736391218808
Epoch 8, Validation Loss: 1.2008344043839585, Accuracy: 0.6842737094837935
Epoch 9, Training Loss: 0.1663998315612284
Epoch 9, V